In [51]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [52]:
# Define the paths to the image folders
benign_path = "C:\\Users\\Manas\\Desktop\\OBJECT DETECTION\\BT\\substracted image\\glioma_substracted\\"
malignant_path = "C:\\Users\\Manas\\Desktop\\OBJECT DETECTION\\BT\\substracted image\\meningioma_substracted\\"
normal_path = "C:\\Users\\Manas\\Desktop\\OBJECT DETECTION\\BT\\substracted image\\pituitary_substracted\\"

In [53]:
# Load the images and extract features
def extract_features(image_path):
    # Read the image
    img = cv2.imread(image_path)
    # Resize the image to a fixed size
    img = cv2.resize(img, (64,64))
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Flatten the image into a 1-dimensional array
    features = gray.flatten()
    return features


In [54]:
# Create a list of all image paths and labels
images = []
labels = []

# Loop through the glioma images and add them to the list
for file_name in os.listdir(benign_path):
    if file_name.endswith(".jpg") or file_name.endswith(".png"):
        image_path = os.path.join(benign_path, file_name)
        features = extract_features(image_path)
        images.append(features)
        labels.append("glioma")
        
        
        
# Loop through the meningioma images and add them to the list
for file_name in os.listdir(malignant_path):
    if file_name.endswith(".jpg") or file_name.endswith(".png"):
        image_path = os.path.join(malignant_path, file_name)
        features = extract_features(image_path)
        images.append(features)
        labels.append("meningioma")

# Loop through the pituitary images and add them to the list
for file_name in os.listdir(normal_path):
    if file_name.endswith(".jpg"):
        image_path = os.path.join(normal_path, file_name)
        features = extract_features(image_path)
        images.append(features)
        labels.append("pituitary")

In [ ]:
# Create a Pandas DataFrame from the images and labels
data = pd.DataFrame(images)
data["label"] = labels


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5023 entries, 0 to 5022
Columns: 4097 entries, 0 to label
dtypes: object(1), uint8(4096)
memory usage: 19.7+ MB


In [7]:
# Split the data into training and testing sets
X = data.drop("label", axis=1)
y = data["label"]



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)



# Apply feature scaling using StandardScaler
scaler = StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
#Create the KNN classifier and fit it to the training data
knn = KNeighborsClassifier(n_neighbors=1, metric='manhattan', algorithm='ball_tree')
knn.fit(X_train, y_train)



KNeighborsClassifier(algorithm='ball_tree', metric='manhattan', n_neighbors=1)

In [9]:
# Predict the labels for the test data
y_pred = knn.predict(X_test)

In [10]:
# Print the classification report
print(confusion_matrix(y_test,y_pred))

[[332  18   0]
 [ 14 297   1]
 [  0   1 342]]


In [11]:
print(classification_report(y_test, y_pred, digits = 5))

              precision    recall  f1-score   support

      glioma    0.95954   0.94857   0.95402       350
  meningioma    0.93987   0.95192   0.94586       312
   pituitary    0.99708   0.99708   0.99708       343

    accuracy                        0.96617      1005
   macro avg    0.96550   0.96586   0.96566      1005
weighted avg    0.96625   0.96617   0.96619      1005



In [12]:
from sklearn.model_selection import train_test_split,cross_val_score
"""X = data.drop("label", axis=1)
y = data["label"]"""

scores = cross_val_score(knn, X, y, cv=5)

In [13]:
print("Average Accuracy: ", np.mean(scores))

Average Accuracy:  0.9100055499395452


In [14]:
print(scores)

[0.95721393 0.89054726 0.92437811 0.89940239 0.87848606]


In [42]:
# Define the paths to the new dataset folders
new_glioma_path = "C:\\Users\\Manas\\Desktop\\OBJECT DETECTION\\BT\\Test\\enhanced image\\glioma_enhanced"
new_meningioma_path = "C:\\Users\\Manas\\Desktop\\OBJECT DETECTION\\BT\\Test\\enhanced image\\meningioma_enhanced"
new_pituitary_path = "C:\\Users\\Manas\\Desktop\\OBJECT DETECTION\\BT\\Test\\enhanced image\\pituitary_enhanced"

In [43]:
# Create a list of all image paths and labels for the new dataset
new_images = []
new_labels = []

# Function to extract features from an image (similar to previous code)
def extract_features(image_path):
    # Read the image
    img = cv2.imread(image_path)
    # Resize the image to a fixed size
    img = cv2.resize(img, (64, 64))
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Flatten the image into a 1-dimensional array
    features = gray.flatten()
    return features

# Function to load images from a folder and append them to the lists
def load_images_from_folder(folder, label):
    for file_name in os.listdir(folder):
        if file_name.endswith(".jpg") or file_name.endswith(".png"):
            image_path = os.path.join(folder, file_name)
            features = extract_features(image_path)
            new_images.append(features)
            new_labels.append(label)
# Load images from each folder in the new dataset and assign labels
load_images_from_folder(new_glioma_path, "glioma")
load_images_from_folder(new_meningioma_path, "meningioma")
load_images_from_folder(new_pituitary_path, "pituitary")

In [44]:
# Create a Pandas DataFrame from the new dataset
new_data = pd.DataFrame(new_images)
new_data["label"] = new_labels

In [45]:
# Separate features and labels for the new dataset
X_new = new_data.drop("label", axis=1)
y_new = new_data["label"]

In [46]:
# Apply the same feature scaling as before
scaler = StandardScaler()
X_new = scaler.fit_transform(X_new)

In [47]:
# Predict the labels for the new dataset
y_pred_new = knn.predict(X_new)

In [48]:
# Print the classification report and accuracy for the new dataset
print("Classification Report on New Dataset:")
print(classification_report(y_new, y_pred_new, digits=5))

Classification Report on New Dataset:
              precision    recall  f1-score   support

      glioma    0.71642   0.48000   0.57485       100
  meningioma    0.65833   0.79000   0.71818       100
   pituitary    0.78761   0.89000   0.83568       100

    accuracy                        0.72000       300
   macro avg    0.72079   0.72000   0.70957       300
weighted avg    0.72079   0.72000   0.70957       300



In [49]:
# You can also print the confusion matrix if needed
print("Confusion Matrix on New Dataset:")
print(confusion_matrix(y_new, y_pred_new))

Confusion Matrix on New Dataset:
[[48 37 15]
 [12 79  9]
 [ 7  4 89]]


In [50]:
# Perform k-fold cross-validation on the new dataset
num_folds = 5  # You can change this to the desired number of folds
scores_new = cross_val_score(knn, X_new, y_new, cv=num_folds)

# Calculate and print the average accuracy
average_accuracy = np.mean(scores_new)
print(f"Average Accuracy on New Dataset (Cross-Validation): {average_accuracy:.5f}")

# You can also print the individual fold scores if needed
print("Cross-Validation Scores on New Dataset:")
print(scores_new)

Average Accuracy on New Dataset (Cross-Validation): 0.74000
Cross-Validation Scores on New Dataset:
[0.8  0.6  0.7  0.75 0.85]
